In [6]:
import csv 
import pandas as pd
from io import StringIO
# import numpy as np
# import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Assume df is your DataFrame containing the pressure data and labels
# df['label'] should be 'Okay' or 'Fault'
# df['prefilling_values'], df['filling_values'], df['stabilizing_values'] should be lists of pressure values


Data1 = []

with open ('minor1.txt') as f:
    reader_obj = csv.reader(f) 
 
    for row in reader_obj:
        Data1.append(row)

csd1 = []
for l in Data1:
    if len(l) == 6:
        # csd.append([l[2],l[3],l[5]])
        if l[2] == "PRF" or l[2] == "FGN" or l[2] == "SDP" or l[2] == "DPD":
            csd1.append([l[2],float(l[3].split()[1]),float(l[5].split()[1]),"okay"])
    
        
# df = pd.DataFrame()
# for d in range(1000):
#     print(csd[d])
            
Prf1 = []
Flg1 = []
Stb1 = []
Dpd1 = []





for d in csd1:
    if d[0] == "PRF":
        Prf1.append([d[2],d[3]])
    elif d[0] == "FGN":
        Flg1.append([d[2],d[3]])
    elif d[0] == "SDP":
        Stb1.append([d[2],d[3]])
    else:
        Dpd1.append([d[2],d[3]])

columns = ['Pressure','label']
Pdf1 = pd.DataFrame(Prf1, columns=columns)
Fdf1 = pd.DataFrame(Flg1, columns=columns)
Sdf1 = pd.DataFrame(Stb1, columns=columns)
Ddf1 = pd.DataFrame(Stb1, columns=columns)

print(Pdf1)
print(Fdf1)
print(Sdf1)
print(Ddf1)

Data2 = []

with open ('minor1.txt') as f:
    reader_obj = csv.reader(f) 
 
    for row in reader_obj:
        Data2.append(row)

csd2 = []
for l in Data2:
    if len(l) == 6:
        # csd.append([l[2],l[3],l[5]])
        if l[2] == "PRF" or l[2] == "FGN" or l[2] == "SDP" or l[2] == "DPD":
            csd2.append([l[2],float(l[3].split()[1]),float(l[5].split()[1]),"fault"])
    
        
# df = pd.DataFrame()
# for d in range(1000):
#     print(csd[d])
            
Prf2 = []
Flg2 = []
Stb2 = []
Dpd2 = []





for d in csd2:
    if d[0] == "PRF":
        Prf2.append([d[2],d[3]])
    elif d[0] == "FGN":
        Flg2.append([d[2],d[3]])
    elif d[0] == "SDP":
        Stb2.append([d[2],d[3]])
    else:
        Dpd2.append([d[2],d[3]])

columns = ['Pressure','label']
Pdf2 = pd.DataFrame(Prf2, columns=columns)
Fdf2 = pd.DataFrame(Flg2, columns=columns)
Sdf2 = pd.DataFrame(Stb2, columns=columns)
Ddf2 = pd.DataFrame(Stb2, columns=columns)

print(Pdf2)
print(Fdf2)
print(Sdf2)
print(Ddf2)

# Convert labels to numerical format
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Create sequences (example: combine pre-filling, filling, stabilizing values)
sequences = df.apply(lambda row: row['prefilling_values'] + row['filling_values'] + row['stabilizing_values'], axis=1)
sequences = sequences.tolist()

# Padding sequences to ensure consistent length
from keras.preprocessing.sequence import pad_sequences
max_seq_length = max(len(seq) for seq in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post', dtype='float32')

# Split data
X = np.array(sequences_padded)
y = df['label'].values
y = to_categorical(y)  # One-hot encode the labels

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Model Design
model = Sequential()
model.add(LSTM(50, input_shape=(max_seq_length, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Training
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluation
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_test_true_classes, y_test_pred_classes))
print(confusion_matrix(y_test_true_classes, y_test_pred_classes))




      Pressure label
0     0.000000  okay
1    -0.000030  okay
2     0.273178  okay
3     1.490379  okay
4     1.707817  okay
...        ...   ...
4695  2.991047  okay
4696  2.991826  okay
4697  2.992914  okay
4698  2.994047  okay
4699  2.995571  okay

[4700 rows x 2 columns]
      Pressure label
0     3.010247  okay
1     3.009940  okay
2     3.009570  okay
3     3.008973  okay
4     3.008132  okay
...        ...   ...
4695  2.993318  okay
4696  2.993245  okay
4697  2.993216  okay
4698  2.993157  okay
4699  2.993164  okay

[4700 rows x 2 columns]
       Pressure label
0      3.002034  okay
1      3.001760  okay
2      3.001772  okay
3      3.001797  okay
4      3.001811  okay
...         ...   ...
18795  2.991887  okay
18796  2.991887  okay
18797  2.991887  okay
18798  2.991887  okay
18799  2.991887  okay

[18800 rows x 2 columns]
       Pressure label
0      3.002034  okay
1      3.001760  okay
2      3.001772  okay
3      3.001797  okay
4      3.001811  okay
...         ...   ...
18